# Workshop 5 - Automate BLAST Runs

### Topics

* About BLAST
* Types of BLAST 
* BLAST Sequence Databases 
* Biopython's BLAST Tools 
    * Running BLAST over the Internet 
    * Saving BLAST output 
    * Parsing BLAST output 
    * Retrieving information from BLAST Record object 
* More Tools in Biopython 
    * Connecting with biological databases 
    * Going 3D
    * Population Genetics
    * Phylogenetics 

## About BLAST 

BLAST (basic local alignment search tool) is an algorithm and program for comparing primary biological sequence information, such as the amino-acid sequences of proteins or the nucleotides of DNA and/or RNA sequences. A BLAST search enables a researcher to compare a subject protein or nucleotide sequence (called a query) with a library or database of sequences, and identify database sequences that resemble the query sequence above a certain threshold. For example, following the discovery of a previously unknown gene in the mouse, a scientist will typically perform a BLAST search of the human genome to see if humans carry a similar gene; BLAST will identify sequences in the human genome that resemble the mouse gene based on similarity of sequence. 

## Types of BLAST

* __Nucleotide-Nucleotide BLAST (blastn)__: given a DNA query, returns the most similar DNA sequences from the DNA database that the user specifies. 
* __Protein-Protein BLAST (blastp)__: given a protein query, returns the most similar protein sequences from the protein database that the user specifies. 
* __Translated Nucleotide-Protein BLAST (blastx)__: for identifying potential protein products encoded by a nucleotide query. 
* __Protein-Translated Nucleotide (tblastn)__: for identifying database sequences encoding proteins similar to the query. 

## BLAST Sequence Databases 

Nucleotide Database:

* __nt (default)__: All GenBank + EMBL + DDBJ + PDB sequences, excluding sequences from PAT, EST, STS, GSS, WGS, TSA and phase 0, 1 or 2 HTGS sequences. Non-redundant, records with identical sequences collapsed into a single entry. 
* __rRNA/ITS databases__: A collection of four databases: a 16S Microbial rRNA sequences from NCBI's Targeted Loci Projects, an 18S and a 26S RNA rRNA databases for fungi, plus an ITS database for fungi. 
* __refseq_rna__: Curated (NM_, NR_) plus predicted (XM_, XR_) sequences from NCBI Reference Sequence Project. 
* ... and more

Protein Databases:

* __nr (default)__: Non-redundant GenBank CDS translations + RefSeq + PDB + SwissProt + PIR + PRF, excluding those in PAT, TSA, and env_nr.
* __refseq_protein__: Protein sequences from NCBI Reference Sequence project. 
* __Landmark__: The landmark database includes proteomes from representative genomes spanning a wide taxonomic range.
* ... and more

For more information about BLAST programs and databases, please see NCBI's [How To BLAST Guide](https://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLASTGuide.pdf).

## Biopython's BLAST Tools 

Most people's first introduction to running BLAST was probably via NCBI web-service. It can be difficult to deal with the volume of data generated by large runs, and to automate BLAST runs in general. Biopython provides several tools to make things easier. We will introduce a few here. 

### Running BLAST over the Internet

This tool allows you to call the online version of BLAST and doesn't need you to download the databases to local. It can save you time and resources if the number of quries you are going to run isn't very large. 

We use the function `qblast()` in the `Bio.Blast.NCBIWWW` module to call the online version of BLAST. This function has three non-optional arguments:

* The first argument is the __blast program__ to use for the search, as a lower case string. Currently `qblast` only works with blastn, blastp, blastx, tblast, and tblastx. 
* The second argument specifies the __databases__ to search against. The options for this are available on the [NCBI Guide to BLAST](https://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLASTGuide.pdf).
* The third argument is a string containing your __query sequence__. This can either be the sequence itself, the sequence in fasta format, or an identifier like a GI number. 

__Usage Guidelines:__

The NCBI BLAST servers are a shared resource. It gives priority to interactive users. In order to ensure availability of the service to the entire community, NCBI may limit searches for some high volume users and will move searches of users who submit more than 100 searches in a 24 hour period to a slower queue, or, in extreme cases, will block the requests. To avoid problems, API users should comply with the following guidelines:

* Do not contact the server more often than once every 10 seconds.
* Do not poll for any single Request ID more often than once a minute. 
* Use the URL parameter email and tool, so that the NCBI can contact you if there is a problem.
* Run scripts weekends or between 9pm and 5am Eastern Time on weekdays if more than 50 searches will be submitted. 

To fulfill the third point, one can set the `NCBIWWW.email` variable.

In [1]:
from Bio.Blast import NCBIWWW

NCBIWWW.email = "u1133824@anu.edu.au" 

The `qblast` function also takes a number of other option arguments, which are basically analogous to the different parameters you can set on the BLAST web page. We'll highlight a few of them here:

* The argument `url_base` sets the base URL for running BLAST over the internet. By default it connects to the NCBI, but one can use this to connect to an instance of NCBI BLAST running in the cloud. 
* The `qblast` function can return the BLAST results in various formats, which you can choose with the optional `format_type` keyword: "HTML", "Text", "ASN.1", or "XML". The default is "XML", as that is the format expected by the parser. 
* The argument `expect` sets the expectation or e-value threshold.

For more about the optional BLAST arguments, you can read the NCBI BLAST's own documentation, or the help documentation built into Biopython:

In [2]:
help(NCBIWWW.qblast)

Help on function qblast in module Bio.Blast.NCBIWWW:

qblast(program, database, sequence, url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', auto_format=None, composition_based_statistics=None, db_genetic_code=None, endpoints=None, entrez_query='(none)', expect=10.0, filter=None, gapcosts=None, genetic_code=None, hitlist_size=50, i_thresh=None, layout=None, lcase_mask=None, matrix_name=None, nucl_penalty=None, nucl_reward=None, other_advanced=None, perc_ident=None, phi_pattern=None, query_file=None, query_believe_defline=None, query_from=None, query_to=None, searchsp_eff=None, service=None, threshold=None, ungapped_alignment=None, word_size=None, short_query=None, alignments=500, alignment_view=None, descriptions=500, entrez_links_new_window=None, expect_low=None, expect_high=None, format_entrez_query=None, format_object=None, format_type='XML', ncbi_gi=None, results_file=None, show_overview=None, megablast=None, template_type=None, template_length=None)
    BLAST search using NCBI's

Note that the default settings on the NCBI BLAST website are not quite the same as the defaults on QBLAST. If you get different results, you'll need to check the parameters (e.g., the expectation value threshold and the gap values). 

__Code Example 01:__ using GI number of your query sequence to search against the nucleotide database (nt) using BLASTN.

In [3]:
result_handle = NCBIWWW.qblast("blastn", "nt", "8332116")

In [ ]:
print(result_handle.getvalue())

The result we get from `qblast` is a StringIO object. A StringIO object is a text stream using an in-memory text buffer. The `getvalue()` method of `StringIO` will return a string containing the entire contents of the buffer. 

__Code Example 02:__ if we have our query sequence already in a FASTA formatted file, we just need to open the file and read in this record as a string, and use that as the query argument:

In [ ]:
fasta_string = open("data/one_cds.fa").read()
result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)

__Code Example 03:__ we could also have read in the FASTA file as a `SeqRecord` object and then supplied just the sequence itself:

### Saving BLAST Output  

Whatever arguments you give the `qblast()` function, you should get back your results in a handle object (by default in XML format). To save a handle object, we can use the following code to save it to file "my_blast.xml". 

In [ ]:
with open("data/my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())

result_handle.close()

`result_handle.close()` closes the handle object and the handle can no longer be read. We can try to get the value of the result_handle again.

In [ ]:
result_handle.getvalue()

### Parsing BLAST Output 

Biopython recommend to parse the output in XML format because it is more stable than the plain text and HTML output and much easier to parse automatically. You can get BLAST output in XML format in various ways. For the parser, it doesn't matter how the output was generated, as long as it is in the XML format. 

To input the XML result from file into Python environment, we use the `open()` function. It will read the file into a handle. In Python, a handle is a nice way of describing input to any info source so that the info can be retrieved using `read()` and `readline()` functions. 

In [ ]:
result_handle = open("data/my_blast.xml")

Now that we have input the XML result into a handle again, we are ready to parse the output. The code to parse BLAST result of a single query is different from the result of multiple queries. 

__Parsing BLAST result for a single query:__

The `NCBIXML.read()` function will read from the handle and store the parsed information in a Blast record object. 

In [ ]:
from Bio.Blast import NCBIXML
blast_record = NCBIXML.read(result_handle)

__Parsing BLAST result for multiple queries:__

The `NCBIXML.parse()` function can return the BLAST result as an iterator, you can use for loop to access each of the BLAST result or store them in a list.

In [ ]:
from Bio.Blast import NCBIXML

result_handle = open("data/two_cds_result.xml")
blast_records = NCBIXML.parse(result_handle)
blast_records_list = list(blast_records)

len(blast_records_list)

__Exercise: please check what attributes and methods does this BLAST record class have?__

### The BLAST Record Class 

A BLAST Record contains everything you might ever want to extract from the BLAST output. Here, we will show an example of how to get some info out of the BLAST report.

To continue with our example result from GI number `8332116`, let's print out some summary info about all hits in our blast report greater than a particular threshold. 

In [ ]:
E_VALUE_THRESH = 0.04

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

Basically, you can do anything you want to with the info in the BLAST report once you have parsed it. You just have to spend more time on understanding the BLAST record object and how to extract the information you need. These objects are defined in `Bio.Blast.Record` and are quite complete. You can find the documentation [here](https://biopython.org/docs/1.75/api/Bio.Blast.Record.html). 

## More Tools in Biopython

Biopython is a very powerful package and it covers a wide range of biological questions. There are many more things we can do using Biopython, and I will introduce a few here. If you would like to know more about Biopython, you can read their [Tutorial and Cookbook](http://biopython.org/DIST/docs/tutorial/Tutorial.html) combined with detailed explanation of Biopython Modules here [API Documentation](https://biopython.org/docs/1.75/api/index.html). 

### Connecting with biological databases 

One of the very common things that we need to do in bioinformatics is extract information from biological databases. It can be quite tedious to access these databases manually, especially if you have a lot of repetitive work to do. Biopython attempts to save you time and energy by making some online databases available from Python scripts. Currently, Biopython has code to extract information from the following databases:

* Entrez (and PubMed) from the NCBI
* ExPASy 
* SCOP 

### Going 3D: The PDB Module

`Bio.PDB` is a Biopython module that focuses on working with crystal structures of biological macromolecules. 

### Bio.PopGen: Population Genetics

`Bio.PopGen` is a Biopython module supporting population genetics. The objective for the module is to support widely used data formats, applications, and databases. `Bio.PopGen` provides a parser and generator of GenePop file format. Utilities to manipulate the content of a record are also provided. 

### Phylogenetics with Bio.Phylo

The `Bio.Phylo` module aims to provide a common way to work with phylogenetic trees independently of the source data format, as well as a consistent API for I/O operations. 

# References

* Biopython - [Biopython Tutorial and Cookbook](http://biopython.org/DIST/docs/tutorial/Tutorial.html) 
* Wikipedia - [BLAST (biotechnology)](https://en.wikipedia.org/wiki/BLAST_(biotechnology))
* NCBI - [How To BLAST Guide](https://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLASTGuide.pdf) 
* NCBI - [BLAST Help Developer Information](https://blast.ncbi.nlm.nih.gov/doc/blast-help/developerinfo.html#developerinfo)